# IBM Instance Setup

In [1]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [3]:
NODES = 18
SP = "Sparse"
MD = "Medium"
DS = "Dense"


ER = "ER"
BP = "Bipartite"
CM = "Complete"

In [3]:
BP18NDParams = np.load("results/params/params_18N_Bipartite_Dense_LR0.00012487564616169122_SIGMA0.10502972406872879_INIT0.6702875946169518_NUMLAYERS3.npy")
BP18NMParams = np.load("results/params/params_18N_Bipartite_Medium_LR0.0017744784691436084_SIGMA1.8303414487358955_INIT0.823301510869179_NUMLAYERS1.npy")
BP18NSParams = np.load("results/archive/params_18N_Bipartite_Sparse_LR0.006312967697938845_SIGMA0.8081435463552646_INIT0.6667394152784892_NUMLAYERS2.npy")
ER18NSParams = np.load("results/archive/params_18N_Bipartite.npy")
ER18NMParams = np.load("results/params/params_18N_ER_Medium_LR0.006024583114622832_SIGMA0.9479329656586278_INIT0.5279665377909606_NUMLAYERS2.npy")
ER18NDParams = np.load("results/params/params_18N_ER_Dense_LR0.0024122980321321335_SIGMA0.8605456674562558_INIT1.001744809410076_NUMLAYERS1.npy")

# Hardware test

In [4]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=156)
print(backend)

<IBMBackend('ibm_aachen')>


In [5]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=512,
                    optimization_level=3)

## Experiments

In [ ]:
QUBITS = NODES * (NODES - 1) // 2
grid_conn = aachen_connectivity()

L1G = efficient_connectivity_gates(grid_conn, QUBITS, 1)
L2G = efficient_connectivity_gates(grid_conn, QUBITS, 2)
L3G = efficient_connectivity_gates(grid_conn, QUBITS, 3)
L4G = efficient_connectivity_gates(grid_conn, QUBITS, 4)
L5G = efficient_connectivity_gates(grid_conn, QUBITS, 5)

L1IQP = iqp.IqpSimulator(QUBITS, L1G, device="lightning.qubit")
L2IQP = iqp.IqpSimulator(QUBITS, L2G, device="lightning.qubit")
L3IQP = iqp.IqpSimulator(QUBITS, L3G, device="lightning.qubit")
L4IQP = iqp.IqpSimulator(QUBITS, L4G, device="lightning.qubit")
L5IQP = iqp.IqpSimulator(QUBITS, L5G, device="lightning.qubit")

@qml.qnode(dev)
def L1Sample(trained_params):
    L1IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L2Sample(trained_params):
    L2IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L3Sample(trained_params):
    L3IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L4Sample(trained_params):
    L4IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

@qml.qnode(dev)
def L5Sample(trained_params):
    L5IQP.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(QUBITS))

650


NameError: name 'dev' is not defined

In [7]:
with Session(backend=backend, max_time="2h") as session:
    ER18NSRes  = L2Sample(ER18NSParams)
    np.save("results/samples/2025-06-07/ER18NS.npy", ER18NSRes)
    ER18NMRes  = L2Sample(ER18NMParams)
    np.save("results/samples/2025-06-07/ER18NM.npy", ER18NMRes)
    ER18NDRes  = L1Sample(ER18NDParams)
    np.save("results/samples/2025-06-07/ER18ND.npy", ER18NDRes)
    BP18NDRes = L3Sample(BP18NDParams)
    np.save("results/samples/2025-06-07/BP18ND.npy", BP18NDRes)
    BP18NMRes = L1Sample(BP18NMParams)
    np.save("results/samples/2025-06-07/BP18NM.npy", BP18NMRes)
    BP18NSRes = L1Sample(BP18NSParams)
    np.save("results/samples/2025-06-07/BP18NS.npy", BP18NSRes)

In [8]:
print("Samples saved successfully.")

Samples saved successfully.
